In [1]:
import pandas as pd
import numpy as np

In [2]:
train_df = pd.read_csv('pfm_train.csv')
test_df = pd.read_csv('pfm_test.csv')

In [3]:
train_df.head(5)
y_train = train_df.Attrition
train_df.drop(['Attrition'],axis=1,inplace=True)

In [4]:
def cleaningdata(data):
    data.drop(['Over18'],axis=1,inplace = True)
    data.drop(['StandardHours'],axis=1,inplace = True)
    data.drop(['EmployeeNumber'],axis=1,inplace = True)
    data = pd.get_dummies(data)
    return data

In [5]:
train_df = cleaningdata(train_df)
test_df = cleaningdata(test_df)

In [8]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train = scaler.fit_transform(train_df)
test_df = scaler.transform(test_df)

In [7]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(train_df, y_train, test_size=0.2)

In [8]:
from sklearn import ensemble
m3 = ensemble.ExtraTreesClassifier(n_estimators=40, 
                                   max_depth=20,
                                   min_samples_split=3,
                                   random_state=0)
m3.fit(x_train, y_train)
y_pred3 = m3.predict(test_df)
sub = pd.DataFrame()
sub['result3'] = y_pred3
# print('Accuracy: {}'.format(m3.score(X_test,y_test)))

In [9]:
from sklearn.svm import SVC
m2 = SVC(C = 2.0)
m2.fit(x_train,y_train)
y_pred2 = m2.predict(test_df)
sub['result2'] = y_pred2

In [10]:
from sklearn import ensemble
m4 = ensemble.RandomForestClassifier(n_estimators=100, 
                                   max_depth=22,
                                   min_samples_split=3,
                                   random_state=0)
m4.fit(x_train, y_train)
y_pred4 = m4.predict(test_df)

sub['result4'] = y_pred4

# print('Accuracy: {}'.format(m4.score(X_test,y_test)))

In [11]:
%%time
from lightgbm import LGBMClassifier
m1 = LGBMClassifier(class_weight='balanced', boosting_type='dart',
                         drop_rate=0.1, min_data_in_leaf=50, 
                         max_bin=255,
                         n_estimators=800,
                         bagging_fraction=0.01,
                         min_sum_hessian_in_leaf=1,
                         importance_type='gain',
                         learning_rate=0.2, 
                         max_depth=10, 
                         num_leaves=10)
m1.fit(x_train, y_train)
y_pred1 = m1.predict(test_df)
sub['result1'] = y_pred1

# print('Accuracy: {}'.format(m1.score(X_test,y_test)))

Wall time: 534 ms


c:\users\user\appdata\local\programs\python\python36\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


## DNN 
### Accuracy **0.8825**
Currently got the best result **0.8825**

In [6]:
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv1D, MaxPool1D, GlobalAveragePooling1D, BatchNormalization
from keras.optimizers import RMSprop, SGD, Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

Using TensorFlow backend.


In [22]:
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler
y = to_categorical(y_train, num_classes=2)
random_seed = 2
# Split the train and the validation set for the fitting
X_train, X_val, Y_train, Y_val = train_test_split(x_train, y, test_size = 0.2, random_state=random_seed)

In [51]:
model = Sequential()
model.add(Dense(32, activation='tanh', input_shape=(48,)))
model.add(Dense(16, activation = "tanh"))
model.add(Dense(16, activation = "tanh"))
# model.add(Dropout(0.5))
model.add(Dense(2, activation = "sigmoid"))

In [52]:
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=1e-04)
optimizer1 = Adam(lr=0.001, decay=1e-4)
optimizer2 = SGD(lr=0.1, decay=1e-4)
model.compile(optimizer = optimizer1 , loss = "categorical_crossentropy", metrics=["accuracy"])
# Set a learning rate annealer
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=0.01, 
                                            verbose=1, 
                                            factor=0.05, 
                                            min_lr=0.0001)
earlystopping = EarlyStopping(monitor='val_acc', patience=0.001, verbose=1, mode='auto')

In [53]:
history = model.fit(X_train, Y_train,
                      epochs=30,
                      batch_size=256,
                      validation_data=(X_val, Y_val),verbose=1)

Train on 880 samples, validate on 220 samples
Epoch 1/30
880/880 [==============================] - 0s 527us/step - loss: 0.6941 - acc: 0.5273 - val_loss: 0.6724 - val_acc: 0.5818
Epoch 2/30
880/880 [==============================] - 0s 25us/step - loss: 0.6701 - acc: 0.5716 - val_loss: 0.6532 - val_acc: 0.6000
Epoch 3/30
880/880 [==============================] - 0s 24us/step - loss: 0.6471 - acc: 0.6216 - val_loss: 0.6333 - val_acc: 0.6636
Epoch 4/30
880/880 [==============================] - 0s 22us/step - loss: 0.6250 - acc: 0.6511 - val_loss: 0.6135 - val_acc: 0.6727
Epoch 5/30
880/880 [==============================] - 0s 23us/step - loss: 0.6024 - acc: 0.6920 - val_loss: 0.5932 - val_acc: 0.7000
Epoch 6/30
880/880 [==============================] - 0s 24us/step - loss: 0.5800 - acc: 0.7159 - val_loss: 0.5727 - val_acc: 0.7227
Epoch 7/30
880/880 [==============================] - 0s 24us/step - loss: 0.5572 - acc: 0.7352 - val_loss: 0.5520 - val_acc: 0.7409
Epoch 8/30
880/880 [==

In [59]:
pred_dnn = np.argmax(model.predict(test_df),axis=1)

In [60]:
sub2 = pd.DataFrame(columns=['result'])
sub2['result'] = pred_dnn
sub2.to_csv('dnn_output.csv',index=None)

### Ensemble Result
voting for the ensemble result

In [12]:
preds = sub.mode(axis=1)

In [13]:
sub2 = pd.DataFrame(columns=['result'])

In [14]:
sub2['result'] = preds[0]
sub2.to_csv('4ensemble_output.csv',index=None)

## SVC Result
#### Accuracy **0.879999**
currently SVC got the best result with 0.879999999

In [20]:
##### SVC got the best result 0.879999999 ########
sub2['result'] = sub['result2']
sub2.to_csv('SVC_output.csv',index=None)

### Feature Importance

In [16]:
pd.DataFrame({'importance':m4.feature_importances_, 'label':train_df.columns.values},
             columns=['importance','label']).sort_values(by='importance',ascending=False)

,importance,label
7,0.079621,MonthlyIncome
0,0.071872,Age
1,0.058607,DistanceFromHome
13,0.056781,TotalWorkingYears
16,0.046280,YearsAtCompany
47,0.042592,OverTime_Yes
9,0.041242,PercentSalaryHike
8,0.038346,NumCompaniesWorked
12,0.033758,StockOptionLevel
46,0.033501,OverTime_No
